### Binary classification based on 3 layers neural network
#### author: Kim Jeong Min

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##### load images

In [0]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.datasets
import torchvision
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pl

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
import argparse
import sys
import os
import numpy as np
import time
import datetime 
import csv
import configparser
import argparse
import platform

from torch.autograd import Variable
import random
from random import shuffle

import math
import torch.utils.data as data_utils

In [0]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)  
        self.conv2 = nn.Conv2d(16, 4, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        
        self.t_conv1 = nn.ConvTranspose2d(4, 16, 2, stride=2)
        self.t_conv2 = nn.ConvTranspose2d(16, 1, 2, stride=2)


    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x) 
        
        x = F.relu(self.t_conv1(x))
        x = F.sigmoid(self.t_conv2(x))
                
        return x

In [0]:
def add_noise(images):
  MEAN = 0
  copyed = np.copy(images)
  for img in copyed:
    sigma = 0.01 * random.randint(1,5)
    gaussian = np.random.normal(MEAN, sigma, (img.shape[0],img.shape[1]))
    img = img + gaussian
  return copyed

In [0]:
transform = transforms.Compose([#transforms.Resize((256,256)),  
                                transforms.Grayscale(),		# the code transforms.Graysclae() is for changing the size [3,100,100] to [1, 100, 100] (notice : [channel, height, width] )
                                transforms.ToTensor(),])


#train_data_path = 'relative path of training data set'
drive_prefix = 'data08/'
train_data_path = drive_prefix + 'train.npy'
train_set = np.load(train_data_path)
# change the valuse of batch_size, num_workers for your program
# if shuffle=True, the data reshuffled at every epoch 
noised = add_noise(train_set)

# make 3d tensor object
train_set = torch.from_numpy(train_set).unsqueeze(1)
noised = torch.from_numpy(noised).unsqueeze(1)

# set feature = noised, target = clean
train_dataset = data_utils.TensorDataset(noised, train_set)
loader_train = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)  

test_data_path = drive_prefix + 'train.npy'
test_set = np.load(test_data_path)
test_set = torch.from_numpy(test_set).unsqueeze(1)

##### load neural network model

In [0]:
model = Autoencoder()

##### Set the flag for using cuda

In [0]:
bCuda = True
if bCuda:
    model.cuda()

##### optimization algorithm

In [0]:
LR = 0.003
optimizer   = optim.SGD(model.parameters(), lr=LR, weight_decay = 1e-5, momentum=0.9)
objective   = nn.MSELoss()

##### function for training the model

In [0]:
def train():
    # print('train the model at given epoch')
    loss_train          = []
    model.train()
    for idx_batch, (data, target) in enumerate(loader_train):
        if bCuda:
            data, target    = data.cuda(), target.cuda()
            
        data, target    = Variable(data), Variable(target)
        optimizer.zero_grad()
        
        output  = model(data)
        loss    = objective(output, target)

        loss.backward()
        optimizer.step()

        loss_train_batch    = loss.item() / len(data)
        loss_train.append(loss_train_batch)
        
    loss_train_mean     = np.mean(loss_train)
    loss_train_std      = np.std(loss_train)

    return {'loss_train_mean': loss_train_mean, 'loss_train_std': loss_train_std}

##### function for testing the model

In [0]:
def test():
    return model(test_set)

##### iteration for the epoch

In [30]:
epoch = 1000
loss_train_mean = list()
loss_train_std = list()

for e in range(epoch):
        
    result_train    = train()
    
    result_loss_train_mean = result_train['loss_train_mean']
    result_loss_train_std =  result_train['loss_train_std']

    loss_train_mean.append( result_loss_train_mean)
    loss_train_std.append(  result_loss_train_std)
    
    epoch_str = '[epoch '+ '{:05d}'.format(e)+ ']  '
    epoch_str += 'loss: '+ '(training)' + str(round(result_loss_train_mean, 15))
    print(epoch_str)
    
    if e > epoch - 100 and accuracy_test[-1] > accuracy_test[-2] and accuracy_test[-1] > accuracy_test[-3]:
        break;
    
    #adaptive learning rate
    for g in optimizer.param_groups:
        g['lr'] = g['lr'] * 0.9985
        
        
result_test = test()

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[epoch 00000]  loss: (training)0.000661924021425
[epoch 00001]  loss: (training)0.000628051349718
[epoch 00002]  loss: (training)0.000620086627601
[epoch 00003]  loss: (training)0.000614340348124
[epoch 00004]  loss: (training)0.000610157225046
[epoch 00005]  loss: (training)0.000611193973927
[epoch 00006]  loss: (training)0.000607534732663
[epoch 00007]  loss: (training)0.000607834676529
[epoch 00008]  loss: (training)0.000606382531207
[epoch 00009]  loss: (training)0.000606847562333
[epoch 00010]  loss: (training)0.000606489398996
[epoch 00011]  loss: (training)0.000607229529018
[epoch 00012]  loss: (training)0.000607062910432
[epoch 00013]  loss: (training)0.000604682530283
[epoch 00014]  loss: (training)0.000603987857703
[epoch 00015]  loss: (training)0.000602898179078
[epoch 00016]  loss: (training)0.000605200998995
[epoch 00017]  loss: (training)0.000606999782115
[epoch 00018]  loss: (training)0.000604375554257
[epoch 00019]  loss: (training)0.000602192781103
[epoch 00020]  loss:

KeyboardInterrupt: ignored

##### plot result

In [0]:
def print_history_graph(loss_history, loss_train_std):
    #plt.plot(loss_history, color='#ff0000', label='Train Loss')
    plt.errorbar(list(range(len(loss_history))), loss_history, yerr=loss_train_std, ecolor = '#ffcccc', color='#ff0000', label='Train Loss')
    plt.plot(vloss_history, color='#0000ff', label='Validation Loss')
    plt.legend(['Validation Loss','Train Loss'])
    plt.title('Loss')
    plt.show()

In [0]:
print_history_graph(loss_train_mean, loss_train_std)